In [12]:
import utils.common
import evaluation
import importlib
import numpy as np
import tensorflow as tf
import models.MatrixFactorization
import models.BPR
import models.ConstraintAutoRec
import models.NeuralLogicRec
import models.MatrixFactorization
import models.BPR
import pandas as pd
from models.NeuralLogicRec import item_cf, user_cf, diversity_constraint, Constraint, And, Or, Implies, Forall, Not, Equiv
importlib.reload(models.MatrixFactorization)

<module 'models.MatrixFactorization' from '/home/ec2-user/SageMaker/rs-ml-with-constraints/models/MatrixFactorization.py'>

# Comparison with baseline models
## MovieLens dataset

In [10]:
eval_ml = evaluation.Evaluation(utils.common.movie_lens)
ev_test = evaluation.Evaluation(utils.common.ml_small)
eval_msd = evaluation.Evaluation(utils.common.msd)

In [8]:
def train_and_evaluate(model, dataset, evaluation):
    evals = []
    model.epochs = 1
    for e in range(12):
        model.train(utils.common.load_dataset(dataset), dataset['train']['records'])
        ev_tmp = evaluation.evaluate_single_thread(model)
        evals.append(ev_tmp)
        print(ev_tmp)
    pd.DataFrame(evals).to_csv('../evals/Comp_' + model.get_name() + '.csv')

In [17]:
constraints = list()
constraints.append(Constraint(weight=0.2, formula=item_cf))
constraints.append(Constraint(weight=0.2, formula=user_cf))
@tf.function
def likes_equiv(model, outputs):
    return Forall(Equiv(outputs['rec'], outputs['likes']))
constraints.append(Constraint(weight=0.95, formula=likes_equiv))
@tf.function
def novelty_constraint(model, outputs):
    return Forall(Implies(outputs['popular'], Not(outputs['rec'])))
constraints.append(Constraint(weight=0.6, formula=novelty_constraint))
constraints.append(Constraint(weight=0.45, formula=diversity_constraint))

In [8]:
nlr_ml = models.NeuralLogicRec.NLR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, constraints=constraints, mode='ae', name='comparison_ML')
train_and_evaluate(nlr_ml, utils.common.movie_lens, eval_ml)
nlr_ml.save('../saved_models/NLR')

Epoch #0 Loss at step 2885: 0.2257, time: 554.524. Train accuracy 0.768, Validation accuracy 0.656
{'accuracy': 0.6416155199349998, 'precision@5': 0.11083720419882229, 'recall@5': 0.046464419736859205, 'map@1': 0.13320653962912843, 'map@5': 0.07065192080270168, 'map@10': 0.05743680360717546, 'diversity@5': 0.12624053053384499, 'diversity@10': 0.1329284142248182, 'epc@5': 0.7303518922017387, 'epc@10': 0.7587401027330339, 'epd@5': 0.15727052481536002, 'coverage@1': 0.033041132838840186, 'coverage@5': 0.07215104517869184, 'coverage@10': 0.09787111068297852, 'name': 'NeuralLogicRec_comparison_ML', 'embedding_dim': 32, 'epochs_trained': 1, 'batch_size': 48, 'nr_hidden_layers': 3, 'nr_item_samples': 4096}
Epoch #0 Loss at step 2885: 0.1895, time: 545.105. Train accuracy 0.842, Validation accuracy 0.691
{'accuracy': 0.7211723235021822, 'precision@5': 0.1425843970593614, 'recall@5': 0.06343103868775374, 'map@1': 0.16129622179144873, 'map@5': 0.09368012630603612, 'map@10': 0.07806333190686732, 

ValueError: If using all scalar values, you must pass an index

In [16]:
car_ml = models.ConstraintAutoRec.ConstraintAutoRec(utils.common.movie_lens['dimensions'], epochs=1, novelty_weight=0.75, diversity_weight=0.75, name='ConstraintAutoRec_ML_pretrain')
train_and_evaluate(car_ml, utils.common.movie_lens, eval_ml)
car_ml.save('../saved_models')

4327/4327 [==============================] - 88s 20ms/step - loss: 0.1797 - accuracy: 0.0102
{'accuracy': 0.8230827624172755, 'precision@5': 0.22994769759701547, 'recall@5': 0.10114069222006004, 'map@1': 0.29113785157821587, 'map@5': 0.17090637179002635, 'map@10': 0.14183405426449297, 'diversity@5': 0.11497341537020579, 'diversity@10': 0.12410851656694785, 'epc@5': 0.7468427317948263, 'epc@10': 0.7620802467038587, 'epd@5': 0.14900602942368313, 'coverage@1': 0.03188517483864753, 'coverage@5': 0.06627492534437915, 'coverage@10': 0.0947885560157981, 'name': 'ConstraintAutoRec_ML_pretrain', 'dimensions': 10381, 'latent_dims': 32, 'accuracy_weight': 1.0, 'novelty_weight': 0.75, 'diversity_weight': 0.75, 'epochs': 1, 'batch_size': 32, 'optimizer': 'adam'}
4327/4327 [==============================] - 87s 20ms/step - loss: 0.1724 - accuracy: 0.0163
{'accuracy': 0.8277965239951341, 'precision@5': 0.23548516879411874, 'recall@5': 0.10485083003383228, 'map@1': 0.29538056398814966, 'map@5': 0.1747

In [23]:
bpr_ml = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048)
train_and_evaluate(bpr_ml, utils.common.movie_lens, eval_ml)
bpr_ml.save('../saved_models')

Epoch #0 Loss at step 6380: 0.6931
{'accuracy': 0.13927232722481883, 'precision@5': 0.0010826231666727626, 'recall@5': 0.000293141763231371, 'map@1': 0.0001828755349109396, 'map@5': 0.00035227923875010665, 'map@10': 0.0002485262623229152, 'diversity@5': 0.1947398687859497, 'diversity@10': 0.17174180196912026, 'epc@5': 0.9947901308589702, 'epc@10': 0.9951814453668391, 'epd@5': 0.19648981028177348, 'coverage@1': 0.0027935651671322607, 'coverage@5': 0.013775166168962528, 'coverage@10': 0.02157788267026298, 'name': 'BPR', 'latent_dim': 128, 'epochs': 1, 'batch_size': 2048}
Epoch #0 Loss at step 6380: 0.6931
{'accuracy': 0.13927232722481883, 'precision@5': 0.0010387330382941372, 'recall@5': 0.00020377206561487905, 'map@1': 0.00025602574887531544, 'map@5': 0.0003923899394072395, 'map@10': 0.0002466990965948575, 'diversity@5': 0.19518281259173512, 'diversity@10': 0.1717969480559117, 'epc@5': 0.9935863801850202, 'epc@10': 0.9945293339671298, 'epd@5': 0.19626259059960452, 'coverage@1': 0.002986

In [11]:
bpr_ml = models.BPR.BPR(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048, latent_dim=40, name='comparison_ml')
train_and_evaluate(bpr_ml, utils.common.movie_lens, eval_ml)
bpr_ml.save('../saved_models')

Epoch #0 Loss at step 6380: 1445.9629
{'accuracy': 0.1405070228874698, 'precision@5': 0.023122782634139208, 'recall@5': 0.006369372858797354, 'map@1': 0.027211879594747815, 'map@5': 0.012752429196688734, 'map@10': 0.009017483720137897, 'diversity@5': 0.17072716962032697, 'diversity@10': 0.17274620463220028, 'epc@5': 0.8849541284035927, 'epc@10': 0.8928357895335679, 'epd@5': 0.17449855518332602, 'coverage@1': 0.17637992486273, 'coverage@5': 0.35690203255948366, 'coverage@10': 0.5003371544167229, 'name': 'BPR_comparison_ml', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch #0 Loss at step 6380: 1380.0099
{'accuracy': 0.5994031876218591, 'precision@5': 0.13507187008522, 'recall@5': 0.05085723744058807, 'map@1': 0.17157382685344355, 'map@5': 0.09327157220129316, 'map@10': 0.0730880774168306, 'diversity@5': 0.11860636120421603, 'diversity@10': 0.11831931334612697, 'epc@5': 0.5730526945444887, 'epc@10': 0.6021639629366432, 'epd@5': 0.15969999853138087, 'coverage@1': 0.014738464502456

In [15]:
mf_ml = models.MatrixFactorization.MatrixFactorization(utils.common.movie_lens['user'], utils.common.movie_lens['dimensions'], epochs=1, batch_size=2048, latent_dim=40, name='comparison_ml')
train_and_evaluate(mf_ml, utils.common.movie_lens, eval_ml)
mf_ml.save('../saved_models')

Epoch 0, loss:  4085.379 |================================================================================>| 100.0% 
{'accuracy': 0.46593181858382865, 'precision@5': 0.0020043158626238985, 'recall@5': 0.0005310334096718402, 'map@1': 0.0021579313119490873, 'map@5': 0.0009683767561131228, 'map@10': 0.0007011750382055051, 'diversity@5': 0.19026801631619011, 'diversity@10': 0.19037210104623184, 'epc@5': 0.9868208759786438, 'epc@10': 0.98687266333382, 'epd@5': 0.1895811787103569, 'coverage@1': 0.5826028320971005, 'coverage@5': 0.8748675464791446, 'coverage@10': 0.9472112513245352, 'name': 'MatrixFactorizationcomparison_ml', 'latent_dim': 40, 'epochs': 1, 'batch_size': 2048}
Epoch 0, loss:  3596.772 |================================================================================>| 100.0% 
{'accuracy': 0.46197927221291163, 'precision@5': 0.001960425734245273, 'recall@5': 0.0005247824529704393, 'map@1': 0.002304231739877839, 'map@5': 0.0009543156594288594, 'map@10': 0.0007030102269122772, 'di

## MSD dataset

In [ ]:
nlr_msd = models.NeuralLogicRec.NLR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, constraints=constraints, mode='ae', name='comparison_MSD')
train_and_evaluate(nlr_msd, utils.common.msd, eval_msd)
nlr_msd.save('../saved_models/NLR')

W0903 08:41:27.533615 140099245623104 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch #0 Loss at step 2457: 0.3172, time: 351.038. Train accuracy 0.758, Validation accuracy 0.482
{'accuracy': 0.47722797440213105, 'precision@5': 0.04563282864493273, 'recall@5': 0.02252205867817557, 'map@1': 0.05786343891021045, 'map@5': 0.02625446932741016, 'map@10': 0.019956593166916232, 'diversity@5': 0.22791160495057736, 'diversity@10': 0.23087719828641146, 'epc@5': 0.9149420019504836, 'epc@10': 0.9224539479450653, 'epd@5': 0.2766878889060244, 'coverage@1': 0.08730631704410012, 'coverage@5': 0.19681168057210965, 'coverage@10': 0.27502979737783073, 'name': 'NeuralLogicRec_comparison_MSD', 'embedding_dim': 32, 'epochs_trained': 1, 'batch_size': 48, 'nr_hidden_layers': 3, 'nr_item_samples': 4096}
Epoch #0 Loss at step 2457: 0.2874, time: 336.456. Train accuracy 0.794, Validation accuracy 0.612
{'accuracy': 0.5730038866342452, 'precision@5': 0.04748850744380245, 'recall@5': 0.023390330997257597, 'map@1': 0.05744169372864915, 'map@5': 0.026605302742749498, 'map@10': 0.020375842757170

In [7]:
bpr_msd = models.BPR.BPR(utils.common.msd['user'], utils.common.msd['dimensions'], epochs=1, batch_size=2048 , latent_dim=40, name='_comparison_msd')
train_and_evaluate(bpr_msd, utils.common.msd, eval_msd)
# bpr_ml.save('../saved_models')

NameError: name 'train_and_evaluate' is not defined